In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
from jax import vmap, jit
import jax.numpy as jnp
from pathlib import Path
import numpy as np
from einops import rearrange
import jax
import matplotlib.pyplot as plt
import seaborn as sns
from hflow.misc.plot import scatter_movie, imshow_movie, plot_grid_movie
from hflow.misc.jax import randkey
from hflow.net.resnet import ResNeSt50FastSmall, ResNet18

In [ ]:
from hflow.data.rwave import get_wave_random_media
sigma = 0
n_x = 128
n_t = 128
sub_x = 4
sub_t = 1
n_samples = 200
key = randkey()
sols = get_wave_random_media(
    n_samples, n_t, n_x, key, batch_size=32, sigma=sigma
)
sols = sols[:, ::sub_t, ::sub_x, ::sub_x]
t_eval = np.linspace(0.0, 1.0, sols.shape[1])
train_mus = np.asarray([0.0])
test_mus = np.asarray([0.0])
mus = np.concatenate([train_mus, test_mus])
sols = jnp.concatenate([sols[None], sols[None]])

In [4]:
net = ResNeSt50FastSmall(n_classes=1, hidden_sizes=[16,32,64])
xx =  jnp.ones((4, 32, 32, 3))
key = randkey()

x_dim = 32 * 32
mu_t_dim = 2
x_in = jnp.zeros(x_dim)
cond_x = jnp.zeros(mu_t_dim)

params_init = net.init(key, x_in, cond_x)

param_count = sum(x.size for x in jax.tree_leaves(params_init))
print(f"n_params {param_count:,}")

out = net.apply(params_init, x_in, cond_x, mutable=False)  
out.shape

n_params 273,025


/tmp/scratch/7607511/ipykernel_183114/3049961609.py:12: DeprecationWarning: jax.tree_leaves is deprecated: use jax.tree.leaves (jax v0.4.25 or newer) or jax.tree_util.tree_leaves (any JAX version).
  param_count = sum(x.size for x in jax.tree_leaves(params_init))


(1,)

In [ ]:
32*32

In [ ]:
sols = rearrange(sols, 'M N T X Y -> ')

In [ ]:
plot_grid_movie(sols[:9])

In [ ]:
from hflow.misc.plot import scatter_movie


scatter_movie(sols[7])

In [ ]:
from hflow.run import build
from hflow.io.load import load_single

cfg, df = load_single('ip', 'ONmD_174258532272')
    
key, train_data, test_data, loss_fn, arg_fn, s_fn, params_init = build(cfg)

In [ ]:
df['train_mus_raw']

In [ ]:
from hflow.misc.jax import randkey

sols_sample, mu_sample, t_sample, quad_weights = arg_fn(randkey())

In [ ]:
(mu_shift, mu_scale) = df ['mu_norm'] 

train_mus , (mu_sample * mu_scale) + mu_shift

In [ ]:
plt.plot(t_sample)
plt.show()

In [ ]:
sols_sample.shape

In [ ]:
scatter_movie(sols_sample)